# Introduction

This notebook trains a linear discriminant analysis model on preprocessed EEG data to predict behavioral data from neural features.

An example of this can be found in the paper, "Human stereoEEG recordings reveal network dynamics of decision-making in a rule-switching task" by Wal et al.

Behavioral data and preprocessed neural data from Subject 06 in the NCSL EFRI datasetwill be used in this notebook.

# Setup

## Imports

In [2]:
import h5py 
import mat73
import numpy as np

## Load File Paths

In [3]:
# ncsl_share = '/run/user/1000/gvfs/smb-share:server=10.162.37.21,share=main'
# data_path = f'Data/Subject06_snapshot_normalized.npy'
subs = ['06']
file_paths = {}

for sub in subs:
    # create a dictionary holding the file paths
    ncsl_share = '/mnt/ncsl_share'
    file_paths[sub] = {
        'setup_path': ncsl_share + f'/Public/EFRI/1_formatted/SUBJECT{sub}/EFRI{sub}_WAR_SES1_Setup.mat',
        'raw_path': ncsl_share + f'/Public/EFRI/1_formatted/SUBJECT{sub}/EFRI{sub}_WAR_SES1_Raw.mat',
        'data_path': ncsl_share + f'/Daniel/Data/Trial_by_Chan_by_Freq_by_Time_Snapshots/Subject{sub}_snapshot_normalized.npy', # movement onset as event
        # 'data_path' : ncsl_share + f'/Daniel/Data/Trial_by_Chan_by_Freq_by_Time_Snapshots/show-card_pre-2sec_post-4sec/Subject{sub}_snapshot_normalized.npy', # visual cue as event
        'out_path_metrics': f'Metrics/Subject{sub}',
        'out_path_plots': f'Plots/Subject{sub}'
    }

In [4]:
raw_file = h5py.File(file_paths['06']['raw_path'])
setup_data = mat73.loadmat(file_paths['06']['setup_path'])

out_path_plots = file_paths['06']['out_path_plots']
out_path_metrics = file_paths['06']['out_path_metrics']

In [5]:
setup_data.keys()

dict_keys(['elec_area', 'elec_ind', 'elec_name', 'filters', 'trial_times', 'trial_words'])

## Instantiate variables

In [5]:
bets = setup_data['filters']['bets']

good_trials = np.where(np.isnan(bets) == False)[0] # extract indices of trials without the 'nan'

bets = bets[good_trials] # get the bet values for the good trials
subject_cards = setup_data['filters']['card1'][good_trials] # get the subject's card values for the good trials

In [6]:
elec_names = np.array(setup_data['elec_name'])
elec_areas = np.array(setup_data['elec_area'])

In [7]:
data = np.load(file_paths['06']['data_path'])
y = np.asarray([(0 if bet == 5 else 1) for bet in bets]) # 0 = low bet ($5), 1 = high bet ($20)

## Matplotlib Settings

In [8]:
import matplotlib as mpl
mpl.rcParams['axes.titlesize'] = 22
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18

## Create Frequency Bands

In [9]:
wavelet_freqs = np.logspace(np.log2(2),np.log2(150),num=63,base=2)

frequency_band_indices ={
    "Delta" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 0.5 and freq < 4],
    "Theta" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 4 and freq < 8],
    "Alpha" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 8 and freq < 14],
    "Beta" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 14 and freq < 30],
    "Gamma" : [i for i,freq in enumerate(wavelet_freqs) if freq >= 30]
}

In [10]:
f_band_data = np.zeros((data.shape[0], data.shape[1], 5, data.shape[3]))

for i, key in enumerate(frequency_band_indices):
    f_band_data[:,:,i,:] = data[:,:,frequency_band_indices[key],:].mean(2)

# Model Training

# Analysis

In [6]:
def find_common_channels(channel_combinations):
    common_channels_dict = {}

    for i in range(len(channel_combinations)):
        for j in range(len(channel_combinations) - i):
            common_channels = []
            if i != i+j:
                common_channels = list(set(channel_combinations[i]).intersection(channel_combinations[i+j]))
                if common_channels != []:
                    common_channels_dict[f'sub_{subs[i]}_and_sub_{subs[i+j]}'] = common_channels
    
    return common_channels_dict

In [7]:
def sort_common_brain_areas(common_channels_dict):
    brain_areas = []
    for areas in common_channels_dict.values():
        for area in areas:
            brain_areas.append(area)
    brain_areas.sort()
    return brain_areas

### Load channel combinations (movment onset)

In [16]:
movement_reg_channel_combinations = []
movement_optimal_channel_combinations = []

subs = ['06','07','10','12','13','15','16','17','18','21']

for sub in subs:
    movement_reg_channel_combinations.append(np.load(f'Metrics/Subject{sub}_optimal_time_window_channel_combination.npy'))
    movement_optimal_channel_combinations.append(np.load(f'Metrics/Subject{sub}_optimal_time_window_optimal_channel_combination.npy'))

### Find common channels in optimal channel combination (movement onset)

In [8]:
movement_common_optimal_channels_dict = find_common_channels(movement_optimal_channel_combinations)

In [33]:
movement_sorted_common_optimal_brain_areas = sort_common_brain_areas(movement_common_optimal_channels_dict)

In [36]:
from collections import Counter
counter = Counter(movement_sorted_common_optimal_brain_areas)

for element, count in counter.items():
    print(f'{element} | Count: {count}')

angular gyrus R | Count: 3
cingulate cortex (posterior) R | Count: 1
fusiform gyrus L | Count: 3
inferior temporal gyrus L | Count: 1
insular cortex (anterior) R | Count: 1
middle temporal gyrus L | Count: 3
middle temporal gyrus R | Count: 1
occipital gyrus L | Count: 3
parietal operculum R | Count: 1
parietooccipital sulcus L | Count: 1
superior temporal gyrus (planum temporale) L | Count: 1
superior temporal gyrus (planum temporale) R | Count: 1
superior temporal sulcus R | Count: 1
supramarginal gyrus L | Count: 1


### Find common channels in regular channel combination (movement onset)

In [17]:
movement_common_reg_channels_dict = find_common_channels(movement_reg_channel_combinations)

In [11]:
movement_common_reg_channels_dict

{'sub_06_and_sub_13': ['fusiform gyrus L'],
 'sub_06_and_sub_16': ['hippocampus (anterior) L'],
 'sub_06_and_sub_17': ['superior temporal gyrus (planum temporale) L',
  'fusiform gyrus L'],
 'sub_06_and_sub_18': ['inferior temporal gyrus L'],
 'sub_06_and_sub_21': ['fusiform gyrus L'],
 'sub_07_and_sub_10': ['inferior frontal gyrus (pars orbitalis) R',
  'angular gyrus R'],
 'sub_07_and_sub_12': ['middle temporal gyrus R',
  'inferior frontal gyrus (pars orbitalis) R',
  'superior temporal gyrus (planum temporale) R',
  'fusiform gyrus R',
  'cingulate cortex (posterior) R',
  'inferior temporal sulcus R',
  'insular cortex (posterior) R',
  'inferior temporal gyrus R'],
 'sub_07_and_sub_13': ['angular gyrus R'],
 'sub_07_and_sub_15': ['supramarginal gyrus R'],
 'sub_07_and_sub_16': ['cuneus R',
  'middle temporal gyrus R',
  'hippocampus (posterior) R',
  'superior temporal gyrus (planum temporale) R',
  'cingulate cortex (posterior) R',
  'angular gyrus R',
  'occipital gyrus R',
  '

In [18]:
movement_sorted_common_reg_brain_areas = sort_common_brain_areas(movement_common_reg_channels_dict)

In [19]:
counter = Counter(movement_sorted_common_reg_brain_areas)

for element, count in counter.items():
    print(f'{element} | Count: {count}')

angular gyrus R | Count: 6
cingulate cortex (posterior) R | Count: 3
cuneus R | Count: 1
fusiform gyrus L | Count: 6
fusiform gyrus R | Count: 1
hippocampus (anterior) L | Count: 1
hippocampus (anterior) R | Count: 1
hippocampus (posterior) R | Count: 1
inferior frontal gyrus (pars opercularis) R | Count: 1
inferior frontal gyrus (pars orbitalis) R | Count: 3
inferior temporal gyrus L | Count: 1
inferior temporal gyrus R | Count: 1
inferior temporal sulcus R | Count: 3
insular cortex (anterior) R | Count: 1
insular cortex (posterior) R | Count: 1
intraparietal sulcus R | Count: 1
middle temporal gyrus L | Count: 10
middle temporal gyrus R | Count: 3
occipital gyrus L | Count: 1
occipital gyrus R | Count: 1
parietal operculum R | Count: 1
parietooccipital sulcus L | Count: 1
superior temporal gyrus (planum temporale) L | Count: 1
superior temporal gyrus (planum temporale) R | Count: 3
superior temporal sulcus R | Count: 1
supramarginal gyrus L | Count: 1
supramarginal gyrus R | Count: 1

In [25]:
list(set(movement_sorted_common_optimal_brain_areas).intersection(movement_sorted_common_reg_brain_areas))

['middle temporal gyrus R',
 'supramarginal gyrus L',
 'insular cortex (anterior) R',
 'superior temporal sulcus R',
 'superior temporal gyrus (planum temporale) L',
 'superior temporal gyrus (planum temporale) R',
 'parietal operculum R',
 'cingulate cortex (posterior) R',
 'inferior temporal gyrus L',
 'angular gyrus R',
 'middle temporal gyrus L',
 'fusiform gyrus L',
 'occipital gyrus L',
 'parietooccipital sulcus L']

### Load channel combinations (visual cue)

In [4]:
vis_reg_channel_combinations = []
vis_optimal_channel_combinations = []

subs = ['06','07','10','12','13','15','16','17','18','21']

for sub in subs:
    vis_reg_channel_combinations.append(np.load(f'Metrics/Subject{sub}_vis_stim_optimal_time_window_channel_combination.npy'))
    vis_optimal_channel_combinations.append(np.load(f'Metrics/Subject{sub}_vis_stim_optimal_time_window_optimal_channel_combination.npy'))

### Find common channels in optimal channel combination (visual cue)

In [8]:
vis_common_optimal_channels_dict = find_common_channels(vis_optimal_channel_combinations)

In [9]:
vis_sorted_common_optimal_brain_areas = sort_common_brain_areas(vis_common_optimal_channels_dict)

In [11]:
from collections import Counter
counter = Counter(vis_sorted_common_optimal_brain_areas)

for element, count in counter.items():
    print(f'{element} | Count: {count}')

angular gyrus R | Count: 1
cingulate cortex (posterior) R | Count: 3
cuneus R | Count: 1
fusiform gyrus L | Count: 3
fusiform gyrus R | Count: 1
hippocampus (posterior) L | Count: 1
insular cortex (posterior) R | Count: 1
intraparietal sulcus R | Count: 1
middle temporal gyrus L | Count: 1
middle temporal gyrus R | Count: 3
occipital gyrus (lateral) L | Count: 1
occipital gyrus L | Count: 1
parietal operculum L | Count: 1
parietooccipital sulcus L | Count: 1
superior temporal gyrus (planum temporale) L | Count: 3
superior temporal gyrus (planum temporale) R | Count: 1
supramarginal gyrus L | Count: 1
temporal pole R | Count: 1


### Find common channel si regular channel combination (visual cue)

In [12]:
vis_common_reg_channels_dict = find_common_channels(vis_reg_channel_combinations)

In [13]:
vis_sorted_common_reg_brain_areas = sort_common_brain_areas(vis_common_reg_channels_dict)

In [15]:
counter = Counter(vis_sorted_common_reg_brain_areas)

for element, count in counter.items():
    print(f'{element} | Count: {count}')

angular gyrus R | Count: 1
cingulate cortex (posterior) R | Count: 6
cuneus R | Count: 1
entorhinal cortex L | Count: 1
fusiform gyrus L | Count: 3
fusiform gyrus R | Count: 1
hippocampus (anterior) L | Count: 1
hippocampus (anterior) R | Count: 1
hippocampus (posterior) R | Count: 1
inferior temporal sulcus R | Count: 1
insular cortex (posterior) R | Count: 1
intraparietal sulcus R | Count: 1
lingula gyrus R | Count: 1
middle temporal gyrus L | Count: 1
middle temporal gyrus R | Count: 6
occipital gyrus L | Count: 1
occipital gyrus R | Count: 1
parietal operculum L | Count: 3
precuneus R | Count: 3
superior temporal gyrus (planum temporale) L | Count: 1
superior temporal gyrus (planum temporale) R | Count: 3
superior temporal sulcus R | Count: 1
supramarginal gyrus L | Count: 3
temporal pole R | Count: 1
